In [23]:
# Extract from https://github.com/StanfordVL/behavior/blob/main/behavior/benchmark/behavior_benchmark.py
import os
import bddl
import json
import behavior
import pyquaternion  
import numpy as np
from PIL import Image
from igibson.utils.utils import parse_config
from igibson.envs.igibson_env import iGibsonEnv
from igibson import object_states

from utils import get_env_config, render_robot_pov

In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
scene_id =  "Benevolence_1_int"
task = "cleaning_out_drawers"
env_config = get_env_config()
env_config["scene_id"] = scene_id
env_config["task"] = task
env_config["task_id"] = 0
env_config["instance_id"] = 0

# Keep this in memory as it's removed from the config when we init iGibson - so we need to reload it
robot_name = env_config["robot"]["name"]

env = iGibsonEnv(
        config_file=env_config,
        mode="headless",
        action_timestep=1.0 / 30.0,
        physics_timestep=1.0 / 120.0,
    )

render_robot_pov(env, env_config, step='initial', show=True, save=False)

																																																																						b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
left_hand_shoulderb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
right_hand_shoulderb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
neckb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identit

/home/nicola/miniconda3/envs/igibson/lib/python3.8/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [26]:
def print_properties(obj):
    """
    Requires from igibson import object_states
    """
    logic_state = {}
    logic_state['InFOVOfRobot'] = obj.states[object_states.InFOVOfRobot].get_value()
    logic_state['InSameRoomAsRobot'] = obj.states[object_states.InSameRoomAsRobot].get_value()
    logic_state['InReachOfRobot'] = obj.states[object_states.InReachOfRobot].get_value()

    print(logic_state)

In [27]:
object_names_list = list(env.task.object_scope.keys())
print("object_names_list: ", object_names_list)

filtered_object_names_list = [name for name in object_names_list \
                              if name.split('.')[0] not in ['agent', 'floor']]
print("filtered_object_names_list: ", filtered_object_names_list)

for object_name in filtered_object_names_list:
    print(f"\nProperties of {object_name}")
    obj = env.task.object_scope[object_name]
    print_properties(obj)

object_names_list:  ['bowl.n.01_1', 'bowl.n.01_2', 'cabinet.n.01_1', 'cabinet.n.01_2', 'spoon.n.01_1', 'spoon.n.01_2', 'piece_of_cloth.n.01_1', 'sink.n.01_1', 'floor.n.01_1', 'agent.n.01_1']
filtered_object_names_list:  ['bowl.n.01_1', 'bowl.n.01_2', 'cabinet.n.01_1', 'cabinet.n.01_2', 'spoon.n.01_1', 'spoon.n.01_2', 'piece_of_cloth.n.01_1', 'sink.n.01_1']

Properties of bowl.n.01_1
{'InFOVOfRobot': False, 'InSameRoomAsRobot': False, 'InReachOfRobot': True}

Properties of bowl.n.01_2
{'InFOVOfRobot': False, 'InSameRoomAsRobot': False, 'InReachOfRobot': True}

Properties of cabinet.n.01_1
{'InFOVOfRobot': False, 'InSameRoomAsRobot': True, 'InReachOfRobot': True}

Properties of cabinet.n.01_2
{'InFOVOfRobot': False, 'InSameRoomAsRobot': True, 'InReachOfRobot': True}

Properties of spoon.n.01_1
{'InFOVOfRobot': False, 'InSameRoomAsRobot': False, 'InReachOfRobot': True}

Properties of spoon.n.01_2
{'InFOVOfRobot': False, 'InSameRoomAsRobot': False, 'InReachOfRobot': True}

Properties of pi